# Using URLs as Context with Gemini

This notebook demonstrates how to provide URLs as context to the Gemini API. The `url_context` tool allows Gemini to extract content from provided URLs as additional context for prompts. This is useful if you want to provide additional uptodate or external context to the model that is not available as part of the internal knowledge of the model.

When `url_context` is enabled the model will use all provided URLs (up to 20) to inform its response. This can be combined with `google_search` to provide a broader context before using the URLs. See below for 2 examples.

Url context is currently in preview without any billing. Quotas are 1500 queries/day/project (API), 100 queries/day/user (AI Studio).

**Supported Models:**
*   `gemini-2.5-pro-preview-05-06`
*   `gemini-2.5-flash-preview-05-20`
*   `gemini-2.0-flash`
*   `gemini-2.0-flash-live-001`

For more details, see the [official documentation](https://ai.google.dev/gemini-api/docs/url-context).


In [ ]:
%uv pip install google-genai --upgrade

**url context, multiple urls**

Will use the `llms.txt` for gemini by example to generate a snippet on how create an image

In [ ]:
import os
from google import genai

# create client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY","xxx"))

# Generate a list of cookie recipes
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-05-20",
    contents='Create a example on how to use Gemini? Using the docs https://ai.google.dev/gemini-api/docs/text-generation https://ai.google.dev/gemini-api/docs/quickstart',
    config={"tools": [{"url_context": {}}]},
)

# print the response
print(f"Response: {response.text}")

Response: The Gemini API allows you to generate text from various inputs, including text, images, video, and audio. To get started, you'll need a Gemini API key, which you can obtain for free from Google AI Studio.

Here's an example of how to use Gemini in Python for text generation:

**1. Install the Google GenAI SDK:**

First, install the necessary library using pip:
`pip install -q -U google-genai`

**2. Make your first request:**

Once the SDK is installed, you can make a request to the Gemini API. Remember to replace `"YOUR_API_KEY"` with your actual API key.

```python
from google import genai

# Configure your API key
genai.configure(api_key="YOUR_API_KEY")

# Choose a model
model = genai.GenerativeModel('gemini-pro') # You can also use 'gemini-2.0-flash' or other available models. [1]

# Generate content
prompt = "Explain how AI works in a few words."
response = model.generate_content(prompt)

# Print the generated text
print(response.text)
```
This example demonstrates a basi

In [28]:
# urls used for grounding
print(f"Context used: {', '.join([url.retrieved_url for url in response.candidates[0].url_context_metadata.url_metadata])}")

Context used: https://ai.google.dev/gemini-api/docs/text-generation, https://ai.google.dev/gemini-api/docs/quickstart


In [29]:
# citations
[response.candidates[0].grounding_metadata.grounding_supports]

[[GroundingSupport(confidence_scores=[1.0], grounding_chunk_indices=[0], segment=Segment(end_index=104, part_index=None, start_index=None, text='The Gemini API allows you to generate text from various inputs, including text, images, video, and audio')),
  GroundingSupport(confidence_scores=[1.0], grounding_chunk_indices=[1], segment=Segment(end_index=203, part_index=None, start_index=106, text="To get started, you'll need a Gemini API key, which you can obtain for free from Google AI Studio")),
  GroundingSupport(confidence_scores=[1.0], grounding_chunk_indices=[1], segment=Segment(end_index=395, part_index=None, start_index=315, text='First, install the necessary library using pip:\n`pip install -q -U google-genai`')),
  GroundingSupport(confidence_scores=[1.0], grounding_chunk_indices=[1], segment=Segment(end_index=1015, part_index=None, start_index=948, text='text)\n```\nThis example demonstrates a basic text generation request')),
  GroundingSupport(confidence_scores=[1.0], groundi

## Combine with Context URL with Google Search


In [45]:
import os
from google import genai

# create client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY","xxx"))

# Generate a list of cookie recipes
response = client.models.generate_content(
    model="gemini-2.5-flash-preview-05-20",
    contents='find the latest versions of the libraries used here and create a pip install command. https://gist.github.com/ultrafunkamsterdam/b1655b3f04893447c3802453e05ecb5e',
    config={"tools": [
        {"url_context": {}},
        {"google_search": {}},
    ]},
)

# print the response
print(f"Response: {response.text}")
# urls used for grounding
print(f"Context used: {', '.join([url.retrieved_url for url in response.candidates[0].url_context_metadata.url_metadata])}")
# searched websited 
print(f"Website used: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")

Response: The latest versions of the libraries are:
*   `fastapi`: `0.115.12`
*   `uvicorn`: `0.34.2`

Here is the `pip install` command:
```bash
pip install fastapi==0.115.12 uvicorn==0.34.2
```
Context used: https://gist.github.com/ultrafunkamsterdam/b1655b3f04893447c3802453e05ecb5e
Website used: FastAPI support for React ( with working react-router ) · GitHub, pypi.org
